# Redis tutorial

## Prerequisites

### Server status

Check the status of your Redis cluster : `!sudo service redis-server status`

If the Redis cluster is inactive, `!sudo service redis-server start` to start it.

In [ ]:
!sudo service redis-server status

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [1]:
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379)

In [3]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [4]:
r.set('greetings', 'hello world')
value = r.get('greetings')
print(value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [5]:
r.set('connections', 0)

True

In [10]:
print("Before:", r.get('connections'))
r.incr('connections')
print("After: ", r.get('connections'))

Before: b'4'
After:  b'5'


In [12]:
print("Before:", r.get('greetings'))
#r.incr('greetings') #doesn't work
#print("After: ", r.get('greetings'))

Before: b'hello world'


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [18]:
listName = 'colors'
r.rpush(listName, 'red')
r.rpush(listName, 'black')
r.rpush(listName, 'blue')

3

**Q** : Display the second element of the list with `LINDEX`

In [20]:
r.lindex(listName, 1)

b'black'

**Q** : Display the full list with `LRANGE`

In [21]:
r.lrange(listName, 0, -1)

[b'red', b'black', b'blue']

### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [22]:
name = "nosql"
r.sadd(name, "redis")
r.sadd(name, "mongodb")
r.sadd(name, "orientdb")
r.sadd(name, "redis")

0

In [24]:
r.smembers(name)

{b'mongodb', b'orientdb', b'redis'}

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [25]:
r.sismember(name, "mysql")

False

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [26]:
r.sadd(name, "hbase")
r.sadd(name, "redis")
r.smembers(name)

{b'hbase', b'mongodb', b'orientdb', b'redis'}

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [32]:
name = "top14"
r.zadd(name, { 'Bordeaux': 10 })
r.zadd(name, { 'Agen': 33 })
r.zadd(name, { 'Brive': 32 })
r.zadd(name, { 'Castres': 29 })
r.zadd(name, { 'Clermont': 38 })
r.zadd(name, { 'Grenoble': 24 })
r.zadd(name, { 'La Rochelle': 26 })
r.zadd(name, { 'Montpellier': 32 })
r.zadd(name, { 'Oyonnax': 14 })
r.zadd(name, { 'Pau': 20 })
r.zadd(name, { 'Racing': 40 })
r.zadd(name, { 'Stade Français': 22 })
r.zadd(name, { 'Toulon': 36 })
r.zadd(name, { 'Toulouse': 36 })

1

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [33]:
print(r.zscore(name, 'Toulon'))
print(r.zrank(name, 'Toulon'))

36.0
10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [35]:
r.zrevrank(name, 'Toulon')

3

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [38]:
r.zrevrange(name, 0, 2, withscores=True)

[(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]

In [40]:
r.zrevrangebyscore(name, 100, 35, withscores=True)

[(b'Racing', 40.0),
 (b'Clermont', 38.0),
 (b'Toulouse', 36.0),
 (b'Toulon', 36.0)]

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [44]:
name = "user"
user = { "id": 1, "name": "Jean", "age": 22 }
r.hmset(name, user)

C:\tools\Anaconda3\envs\utec-bigdata-2022-1\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


True

In [45]:
r.hgetall(name)

{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [46]:
r.hset(name, "city", "Lyon")

1

In [47]:
r.hgetall(name)

{b'id': b'1', b'name': b'Jean', b'age': b'22', b'city': b'Lyon'}

In [48]:
r.hset(name, "name", "Paul")
r.hgetall(name)

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

**Q** : Create a set of cache request keys.

**Q** : Check if the GET request on http: //http://my-api.fr/user

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [ ]:
!redis-cli flushall